# COVIDx-CT Dataset Constructor
This notebook constructs the COVIDx-CT dataset from scratch using the raw data. See [docs/dataset.md](docs/dataset.md) for more details on manual steps which must be completed beforehand.

In [1]:
import os
import cv2
import glob
import numpy as np

## Setting Paths
Paths to the source data and output location should be set here. Note that the window width and window level for processing scans in Hounsfield units are defined in [data_utils.py](data_utils.py) as `HU_WINDOW_WIDTH = 1500` and `HU_WINDOW_LEVEL = -600`.

In [2]:
# Select dataset version. Major version (e.g., v2) should match current release.
# Checkout earlier versions of the repo to construct previous dataset versions.
DATASET_VERSION = 'v2A'

# CNCB AI Diagnosis paths
CNCB_EXCLUDE_FILE = 'dataset_construction/metadata/cncb_exclude_list.txt'
CNCB_DIR = 'data/CNCB_AIDiagnosis'

# Radiopaedia/coronacases segmentation data paths
RADIOPAEDIA_CORONACASES_CT_DIR = 'data/Coronacases_Radiopaedia/COVID-19-CT-Seg_20cases'
RADIOPAEDIA_CORONACASES_SEG_DIR = 'data/Coronacases_Radiopaedia/Infection_Mask'

# LIDC-IDRI paths
LIDC_META_CSV = 'dataset_construction/metadata/lidc_idri_metadata.csv'

# COVID-19-20 paths
COVID_19_20_DIR = 'data/COVID-19-20_v2/Train'

# COVID-CTset paths
COVID_CTSET_META_CSV = 'data/COVID-CTset/Labels&Detailes/Patient_details.csv'
COVID_CTSET_DIR = 'data/COVID-CTset/Train&Validation'

# MosMedData paths
MOSMED_CT_DIR = 'data/MosMedData/COVID19_1110/studies/CT-1'
MOSMED_SEG_DIR = 'data/MosMedData/COVID19_1110/masks'

# Extra Radiopaedia data paths
RADIOPAEDIA_META_CSV = 'dataset_construction/metadata/radiopaedia_metadata.csv'
RADIOPAEDIA_EXCLUDE_FILE = 'dataset_construction/metadata/radiopaedia_exclude_list.txt'

# Extra TCIA COVID-19 study paths
TCIA_COVID_META_CSV = 'dataset_construction/metadata/tcia_covid_metadata.csv'
TCIA_COVID_EXCLUDE_FILE = 'dataset_construction/metadata/tcia_covid_exclude_list.txt'
TCIA_DIR = 'data/CT_Images_in_COVID-19_August_2020'

# Output directory path
OUTPUT_DIR = 'data/COVIDx-CT_{}'.format(DATASET_VERSION)  # directory to save the images in

# Make output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

## Data Preparation
These cells process the data from each source.

In [3]:
# Filename and class lists
filenames, classes = [], []

In [4]:
# Process CNCB AI Diagnosis data
from dataset_construction import cncb
fnames, cls = cncb.process_cncb_data(CNCB_EXCLUDE_FILE, CNCB_DIR, OUTPUT_DIR)
filenames.extend(fnames)
classes.extend(cls)

100%|███████████████████████████████████████████████████████████████████████| 103981/103981 [00:03<00:00, 26829.40it/s]


In [5]:
# Process radiopaedia and coronacases data
from dataset_construction import radiopaedia_coronacases as rc
fnames, cls = rc.process_radiopaedia_and_coronacases_seg_data(
    RADIOPAEDIA_CORONACASES_CT_DIR, RADIOPAEDIA_CORONACASES_SEG_DIR, OUTPUT_DIR)
filenames.extend(fnames)
classes.extend(cls)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:43<00:00,  2.15s/it]


In [6]:
# Process LIDC-IDRI data
from dataset_construction import lidc_idri
fnames, cls = lidc_idri.process_lidc_idri_data(LIDC_META_CSV, OUTPUT_DIR)
filenames.extend(fnames)
classes.extend(cls)

100%|██████████████████████████████████████████████████████████████████████████████████| 39/39 [01:47<00:00,  2.76s/it]


In [7]:
# Process COVID-19-20 challenge data
from dataset_construction import covid_19_20
fnames, cls = covid_19_20.process_covid_19_20_data(COVID_19_20_DIR, OUTPUT_DIR)
filenames.extend(fnames)
classes.extend(cls)

100%|████████████████████████████████████████████████████████████████████████████████| 199/199 [03:17<00:00,  1.01it/s]


In [8]:
# Process COVID-CTset data
from dataset_construction import covid_ctset
fnames, cls = covid_ctset.process_covid_ctset_data(COVID_CTSET_META_CSV, COVID_CTSET_DIR, OUTPUT_DIR)
filenames.extend(fnames)
classes.extend(cls)

100%|████████████████████████████████████████████████████████████████████████████████| 377/377 [00:08<00:00, 45.69it/s]


In [9]:
# Process MosMedData studies (only for "B" variant)
if DATASET_VERSION[-1] == 'B':
    from dataset_construction import mosmed
    fnames, cls = mosmed.process_mosmed_data(MOSMED_CT_DIR, MOSMED_SEG_DIR, OUTPUT_DIR)
    filenames.extend(fnames)
    classes.extend(cls)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:29<00:00,  1.72it/s]


In [10]:
# Process extra Radiopaedia studies (only for "B" variant)
if DATASET_VERSION[-1] == 'B':
    from dataset_construction import radiopaedia
    fnames, cls = radiopaedia.process_radiopaedia_data(RADIOPAEDIA_META_CSV, RADIOPAEDIA_EXCLUDE_FILE, OUTPUT_DIR)
    filenames.extend(fnames)
    classes.extend(cls)

100%|████████████████████████████████████████████████████████████████████████████████| 118/118 [00:45<00:00,  2.60it/s]


In [11]:
# Process extra TCIA COVID-19 studies (only for "B" variant)
if DATASET_VERSION[-1] == 'B':
    from dataset_construction import tcia_covid
    fnames, cls = tcia_covid.process_tcia_covid_data(TCIA_COVID_META_CSV, TCIA_COVID_EXCLUDE_FILE, TCIA_DIR, OUTPUT_DIR)
    filenames.extend(fnames)
    classes.extend(cls)

100%|████████████████████████████████████████████████████████████████████████████████| 248/248 [03:20<00:00,  1.24it/s]


In [12]:
# Print image counts
from dataset_construction.utils import CLASS_MAP
uniq_classes, counts = np.unique(classes, return_counts=True)
print('Image Counts')
for name, cls in CLASS_MAP.items():
    print('{}: {}'.format(name, counts[uniq_classes == cls]))

Image Counts
Normal: [60083]
Pneumonia: [38121]
COVID-19: [35716]


## Verification
Perform a check to ensure that all files are present (optional).

In [14]:
# Get split files
split_files = glob.glob(os.path.join('splits/' + DATASET_VERSION[:2], '*_COVIDx-CT_{}.txt'.format(DATASET_VERSION)))
if not len(split_files):
    raise ValueError('Split files for COVIDx-CT {} not found'.format(DATASET_VERSION))

# Default to "A" variant when "B" variant files are not present
if len(split_files) != 3:
    a_var = DATASET_VERSION[:-1] + 'A'
    existing_splits = set(os.path.basename(split_file).split('_')[0] for split_file in split_files)
    split_files_a = glob.glob(os.path.join('splits/' + DATASET_VERSION[:2], '*_COVIDx-CT_{}.txt'.format(a_var)))
    split_files_a = [f for f in split_files_a if os.path.basename(f).split('_')[0] not in existing_splits]
    split_files = split_files + split_files_a

# Check that all files from all splits are present in the constructed data
count = 0
total = 0
incomplete = False
for split_file in split_files:
    with open(split_file, 'r') as f:
        for line in f.readlines():
            fname, cls = line.strip('\n').split()[:2]
            fpath = os.path.join(OUTPUT_DIR, fname)
            
            total += 1
            if os.path.exists(fpath):
                count += 1
            else:
                print('Missing', fname)
                incomplete = True
if incomplete:
    print('{}/{} files are missing, dataset is incomplete!'.format(count, total))
else:
    print('{}/{} files created, dataset successfully constructed!'.format(count, total))

133920/133920 files created, dataset successfully constructed!
